# Concat Operator with caching

* https://grantjenks.com/docs/diskcache/
* https://joblib.readthedocs.io/en/latest/index.html
* https://xarray.pydata.org/en/stable/generated/xarray.tutorial.open_dataset.html#xarray.tutorial.open_dataset
* https://www.fatiando.org/pooch/latest/
* https://pypi.org/project/appdirs/

In [1]:
import xarray as xr
import pandas as pd
from pathlib import Path
from diskcache import Cache
import tempfile
import time

In [2]:
def _concat(dataset, dim=None):
    data_dir = Path(dataset)
    paths = sorted(data_dir.glob("**/*.nc"))

    # aggregation
    ds_avg = None
    with xr.open_mfdataset(
        paths,
        concat_dim=dim,
        combine="nested",
        chunks={dim: 10, "time": 10},
        # parallel=True,
        # preprocess=lambda ds: ds.isel(time=0)
    ) as ds:
        # average
        ds_avg = ds.mean(dim="realization", skipna=True, keep_attrs=True)
    return ds_avg

In [3]:
def concat(dataset, dim=None, cache=True, cache_dir=None):
    dim = dim or "realization"
    dcache = Cache(cache_dir)
    key = f"concat_{dim}_{dataset}"
    print(key)
    print("cache stats: ", dcache.stats(enable=True))
    if cache and key in dcache:
        print("use cache")
        ds_path = dcache.get(key)
        ds = xr.open_dataset(ds_path)
    else:
        print("concat")
        ds = _concat(dataset, dim)
        tempdir = tempfile.mkdtemp(dir=dcache.directory, prefix="concat_") 
        ds_path = Path(f"{tempdir}/out.nc").as_posix()
        print("write netcdf", ds_path)
        ds.to_netcdf(ds_path)
        print("add to cache")
        dcache.set(key, ds_path, expire=None, tag="concat")
    return ds 

In [4]:
start = time.time()
dataset = "/Users/pingu/data/cmip6-decadal/orig/day"
ds = concat(dataset, cache_dir="/Users/pingu/data/cache")
print(f"{time.time() - start} secs")

concat_realization_/Users/pingu/data/cmip6-decadal/orig/day
cache stats:  (2, 0)
use cache
0.3773081302642822 secs


In [5]:
ds

<xarray.Dataset>
Dimensions:   (time: 3714, lat: 192, bnds: 2, lon: 384)
Coordinates:
  * time      (time) datetime64[ns] 1970-11-01T12:00:00 ... 1980-12-31T12:00:00
  * lat       (lat) float64 -89.28 -88.36 -87.42 -86.49 ... 87.42 88.36 89.28
  * lon       (lon) float64 0.0 0.9375 1.875 2.812 ... 356.2 357.2 358.1 359.1
    height    float64 ...
Dimensions without coordinates: bnds
Data variables:
    lat_bnds  (lat, bnds) float64 ...
    lon_bnds  (lon, bnds) float64 ...
    tas       (time, lat, lon) float32 ...
Attributes: (12/39)
    Conventions:           CF-1.7 CMIP-6.2
    activity_id:           DCPP
    branch_method:         standard lagged initialization
    contact:               cmip6-mpi-esm@dkrz.de
    creation_date:         2021-04-05T18:46:24Z
    data_specs_version:    01.00.30
    ...                    ...
    title:                 MPI-ESM1-2-HR output prepared for CMIP6
    variable_id:           tas
    variant_label:         r10i1p1f1
    license:               CMIP6 model data produced by MPI-M is licensed und...
    cmor_version:          3.5.0
    tracking_id:           hdl:21.14100/fd0fba88-98dc-4ccc-bdb9-46573c7d30d3